In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cpu


In [2]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)


['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [3]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])


In [4]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)



inputs:
tensor([[73, 54, 64, 58, 67,  9,  3,  1],
        [59, 68, 68, 65, 62, 72, 61,  1],
        [68, 67, 58,  1, 73, 61, 54, 73],
        [74,  1, 72, 54, 78,  1, 62, 73]])
targets:
tensor([[54, 64, 58, 67,  9,  3,  1, 57],
        [68, 68, 65, 62, 72, 61,  1, 73],
        [67, 58,  1, 73, 61, 54, 73,  1],
        [ 1, 72, 54, 78,  1, 62, 73,  1]])


In [5]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [6]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



(sFAWMnw_4hh7dXP_AsvR?﻿D[wbs 5j﻿f6!;sAG(!
xy]7G4Cr4", Jy.S_fJ_q]i1]BQ;FtS;H ;DrjSyU2.zn6tc,ZPqJfG&*s4,:**8z0cp;wjo&t
rm?,Y!nqog5tymN1VKk*﻿atv)I]W!HXjSDfyHgQXqRgCGejmNb[_clyP-esIClV8"0LmO0c(r0Cklyu6tS'?﻿x)T1]fgkzIz]_mk5tiWvuv7O4a5mG&8&IT
﻿oiXUm"OTPh1L(aeRsh:Qn"BUB2i
nkEXs.7YR2sFM14JopdFb 7[vB*O
uY?,)pdJ)U2QLm;:")rOF,.lpdwLVd8uSb7avXwrm( nocdJ8Y[&ZI;e"ne_)"9QA)NB1!w2OEAow0-VbhRN7VbH9zz4Yf;X&)rm9I(K46PDw]cnx﻿zVVb8R
8L4﻿*Vi
JZw7lWfl-c45.JLa1]H&hnn'_"LW:ozrmS[.PZ,)RUzKUimXUu_.V;S﻿*93o?[g_5BHFtH8u1E5F


In [7]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 5.045, val loss: 5.047
step: 250, train loss: 4.993, val loss: 4.982
step: 500, train loss: 4.919, val loss: 4.900
step: 750, train loss: 4.838, val loss: 4.862
4.635464668273926


***need to familiarize audience with optimizers (AdamW, Adam, SGD, MSE…) no need to jump into the formulas, just what the optimizer does for us and some of the differences/similarities between them***

1. **Mean Squared Error (MSE)**: MSE is a common loss function used in regression problems, where the goal is to predict a continuous output. It measures the average squared difference between the predicted and actual values, and is often used to train neural networks for regression tasks.
2. **Gradient Descent (GD):**  is an optimization algorithm used to minimize the loss function of a machine learning model. The loss function measures how well the model is able to predict the target variable based on the input features. The idea of GD is to iteratively adjust the model parameters in the direction of the steepest descent of the loss function
3. **Momentum**: Momentum is an extension of SGD that adds a "momentum" term to the parameter updates. This term helps smooth out the updates and allows the optimizer to continue moving in the right direction, even if the gradient changes direction or varies in magnitude. Momentum is particularly useful for training deep neural networks.
4. **RMSprop**: RMSprop is an optimization algorithm that uses a moving average of the squared gradient to adapt the learning rate of each parameter. This helps to avoid oscillations in the parameter updates and can improve convergence in some cases.
5. **Adam**: Adam is a popular optimization algorithm that combines the ideas of momentum and RMSprop. It uses a moving average of both the gradient and its squared value to adapt the learning rate of each parameter. Adam is often used as a default optimizer for deep learning models.
6. **AdamW**: AdamW is a modification of the Adam optimizer that adds weight decay to the parameter updates. This helps to regularize the model and can improve generalization performance. We will be using the AdamW optimizer as it best suits the properties of the model we will train in this video.

find more optimizers and details at torch.optim

In [8]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


(8MFN9HFA:ap8clPc-Q3A?o&ogK8";bAzrELp*6tiBgiL6wx_]p8u'f[4YG]_9,_J?PPlKDBH[!﻿or﻿"]OJEX"
jlVbaeQ;2pzKn)'fJq5TmxiLpG_
([3CMC*﻿0c8EY[2hoBXPVGEJGu"M
Rshq XYS-r[
hFA:2jRYqmX'I&h.f_MgAvF:C
([T?﻿vQn]7U6kEdvMPjU6t&rmWv'﻿GzK!mGtuT6:6w*T!"X&z?&?y9Y3opc"D73S*5']0o0a'iW7KU!
jG!U
[
J
_
j16kxXLn
hsup e8T-kCKOej]R1;NN]wTIzwA)44X.)g7jMP8pv)LtI)"F5N8GKgw0kR(1J_O((t
_9xnFbSiSeGFUudIrmt_StkF:9.[D:dq"9a
;3Euowkco21AGsjsux)pU2s7bUIG[fJy)VZl?Mn'oEEj'M_KRE5kzrm[.vMpd"DeyHIqYsubAFR9KX3CEMP 
([UujhtinsI.PIC1A -nU[j]cgn]C
